In [1]:
import numpy
import time
from datetime import datetime, timedelta
import scipy.io
import csv
import os.path

from nsopy.methods.subgradient import SubgradientMethod
from nsopy.loggers import GenericMethodLogger

# Author: Renzo Caballero
# KAUST: King Abdullah University of Science and Technology
# email 1: renzo.caballerorosas@kaust.edu.sa
# email 2: CaballeroRenzo@hotmail.com
# email 3: CaballeroRen@gmail.com
# Website: None
# November 2019; Last revision: 07/12/2019

In [2]:
def oracle(x):
    
    # We load data from the previous optimization:
    path = ['Simulations/table_',actualDate,'.mat']
    path = "".join(path)
    mat = scipy.io.loadmat(path)
    array = mat.get('saveTable')
    lastRow = array[-1]
    auxSft = lastRow[0:4]
    sft = numpy.array([val for val in auxSft for _ in (0, 1)])
     
    if len(x) == 1:
        x = x[0]
    
    comm = numpy.asarray([x[0]+sft[0],x[1]+sft[1],x[2]+sft[2],x[3]+sft[3], # 4
                          x[4]+sft[4],x[5]+sft[5],x[6]+sft[6],x[7]+sft[7], # 4
                          0,0,0,0,0,0,0,0,0,0]);
    numpy.savetxt("loopForOpt_2.csv", comm, delimiter=",");
    done = 0;

    while done == 0:

        time.sleep(20);
        comm = numpy.loadtxt("loopForOpt_2.csv", delimiter =', ');

        if float(comm[-1]) == 1:
            print(comm[0:8]);
            print(comm[9:17]);
            print(comm[8]);
            done = 1;
        else:
            print('Waiting for MATLAB...');
            
    f_x_k = comm[8];
    diff_f_xk = [comm[9:17]];
    
    return 0, f_x_k, numpy.array(diff_f_xk)

def projection_function(x_k):
    if x_k is 0:
        return numpy.array([0,])
    else:
        return numpy.maximum(x_k, 0)

In [3]:
#initialDate = "20180609"
initialDate = "20180405"
finalDate = "20191130"
dateStringFormat = '%Y%m%d'

actualDate = initialDate

while actualDate != finalDate:
    
    path = ['Simulations/table_',actualDate,'.mat']
    path = "".join(path)
    path2 = ['Simulations/table_2_',actualDate,'.mat']
    path2 = "".join(path2)
    
    # If table 1 exists and table 2 does not, then we compute table 2:
    if os.path.isfile(path) and not(os.path.isfile(path2)):
    
        print("".join(['We will compute day: ',actualDate]))
    
        # Create .csv file with the date that we will compute:
        myFile = open('date_2.csv', 'w')
        with myFile:
            writer = csv.writer(myFile)
            writer.writerows([[actualDate]])

        # Create path and load .mat file:
        path = ['Historical/dailyData/Day_',actualDate,'.mat']
        path = "".join(path)
        mat = scipy.io.loadmat(path)
        # We use only the dams costs and create the step:
        array = mat.get("Matrix")
        array = array[0]
        array = array[2]
        div = 0.05
        step = numpy.mean(array) / div
        # If all dams have value zero, we set the step to 1:
        if step == 0:
            step = 1

        method = SubgradientMethod(oracle, projection_function, dimension=8, stepsize_0=step, stepsize_rule="1/k", sense='min')
        logger = GenericMethodLogger(method)

        for iteration in range(5):
            method.step()
        
    # We increment the actualDate in one day:
    actualDate = datetime.strptime(actualDate, dateStringFormat)
    actualDate = actualDate + timedelta(days=1)
    actualDate = actualDate.strftime(dateStringFormat)


We will compute day: 20180516
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
[0.0000e+00 0.0000e+00 1.1382e+00 1.1382e+00 2.9686e-15 2.9686e-15
 4.4782e+00 4.4782e+00]
[ 2.1820e-17  3.5936e-17  5.9625e-17  7.1223e-02 -9.3132e-17 -0.0000e+00
 -1.8626e-16 -4.2991e-01]
73.028
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
[0.0000e+00 0.0000e+00 1.1382e+00 1.1382e+00 1.2282e-14 2.9686e-15
 4.4782e+00 4.7469e+01]
[ 6.5403e-17  2.5675e-17  1.1771e-16  7.1282e-02 -2.3283e-16 -0.0000e+00
 -9.3132e-17 -1

Waiting for MATLAB...
Waiting for MATLAB...
[  0.     0.     0.     0.   232.08 232.06  12.28  12.28]
[1.1106  1.1107  1.1107  1.0181  2.1877  2.0532  0.97776 1.1958 ]
-143.02
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
[ 0.     0.     0.     0.    22.607 35.466 12.28  12.28 ]
[1.1106  1.1107  1.1107  1.0181  0.97585 0.97595 0.97638 1.1941 ]
-149.96
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
[0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 2.4549e-15 2.4549e-15
 1.2280e+01 1.2280e+01]
[ 1.1106   1.1107   1.1107   1.0181  -1.2119  -1.2118   0.97638  1.1941 ]
-150.48
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
[ 0.     0.     0.     0.    58.02  58.015 12.28  12.28 ]
[1.1106  1.1107  1.1107  1.0181  0.97585 0.97595 0.97638 1.1941 ]
-149.44
We will compute day: 20180716
Waiting for MATLAB...
Waiti

[125.63  125.63   48.899  48.899 143.07  143.07  142.67  142.67 ]
[-1.1319   0.34402 -1.132    0.62537  1.0571   1.057    1.0572   1.2923 ]
123.93
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
[739.12  125.63  662.44   48.899 143.07  143.07  142.67  142.67 ]
[2.1279  0.25142 2.1279  0.62568 1.0571  1.057   1.0567  1.2914 ]
142.73
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
[162.46  125.63   85.782  48.899 143.07  143.07  142.67  142.67 ]
[-0.20594  0.15883 -0.20611  0.62537  1.0571   1.057    1.0571   1.292  ]
123.32
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
[199.67  125.63  123.02   48.899 143.07  143.07  142.67  142.67 ]
[1.562   0.15778 1.0902  0.62544 1.0571  1.057   1.0567  1.2914 ]
123.46
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
[125.63  125.63   48.899  48.899

Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
[ 0.     0.     0.     0.    24.075 24.075 75.715 75.715]
[ 1.6959e-04  1.8470e-04  1.2663e-01 -1.3044e-01  2.1879e+00  2.1879e+00
  2.1879e+00  2.6741e+00]
307.52
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
[ 0.     0.     0.    27.501 24.075 24.075 75.715 75.715]
[3.6996e-04 3.7653e-04 2.8872e-01 2.2428e+00 2.1879e+00 2.1879e+00
 2.1879e+00 2.6741e+00]
308.03
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
[ 0.     0.     0.     0.    24.075 24.075 75.715 75.715]
[ 1.8557e-04  1.8922e-04  1.2645e-01 -1.3083e-01  2.1879e+00  2.1879e+00
  2.1879e+00  2.6741e+00]
307.52
We will compute day: 20180822
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
[ 0.      0.      0.      0.     18.852  18.852   9.4069  9.4069]
[-0.017502 -0.0175   -0.

Waiting for MATLAB...
[0.9994  0.9724  0.7689  0.      0.      0.74755 7.6899  7.6899 ]
[ 2.2222   2.2222   2.2222   1.1111  -0.85629  2.0779   2.0779   2.5397 ]
0.3932
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
[0.25867  0.23167  0.028167 0.       0.28543  0.054917 7.6899   7.6899  ]
[2.2222 2.2222 2.2221 1.111  2.0779 2.0779 2.0779 2.5397]
0.3319
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
[0.     0.     0.     0.     0.     0.     7.6899 7.6899]
[-2.1105   -2.0835   -1.88     -0.015131 -0.85668  -1.7865    2.0779
  2.5397  ]
0.32033
We will compute day: 20180909
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
[0. 0. 0. 0. 0. 0. 0. 0.]
[-2.1105   -2.1077    0.14061  -0.022683 -1.5493   -1.3172   -2.0058
 -2.4324  ]
6.2074e-14
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
W

Waiting for MATLAB...
Waiting for MATLAB...
[0. 0. 0. 0. 0. 0. 0. 0.]
[-1.1416 -1.1417 -1.1417 -1.2369 -1.5767 -1.3545 -1.1172 -1.3652]
-28.442
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
[1.7124 1.7125 1.7125 1.8553 2.365  2.0317 1.6758 2.0478]
[-1.1440e+00 -1.1451e+00 -1.1438e+00 -1.0539e+00 -5.0244e-17  1.9673e-17
  8.9090e-01  1.3183e+00]
-55.174
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
[2.5704 2.5714 2.5704 2.6458 2.365  2.0317 1.0076 1.0591]
[-1.3970e-16 -4.7649e-01 -1.1435e+00 -1.0536e+00  1.3552e-16  6.8687e-17
  8.9096e-01  1.3183e+00]
-55.202
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
[2.5704  2.8096  3.1421  3.1726  2.365   2.0317  0.56214 0.39992]
[-0.0000e+00 -3.8240e-01 -1.1435e+00 -1.05

Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
[0.9845  0.9845  0.9842  0.6852  0.33626 0.33626 0.22832 0.22832]
[2.2222 2.2222 2.2222 2.4074 2.0957 2.0957 2.0957 2.5614]
0.10086
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
[0.24377 0.24377 0.24347 0.      0.33626 0.33626 0.22832 0.22832]
[2.2222  2.2222  2.2222  0.18502 2.0957  2.0957  2.0957  2.5614 ]
0.037993
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
[0.      0.      0.      0.      0.33626 0.33626 0.22832 0.22832]
[-2.0956 -2.0956 -2.0953 -1.8889  2.0957  2.0957  2.0957  2.5614]
0.022929
We will compute day: 20181009
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
[0. 0. 0. 0. 0. 0. 0. 0.]
[ 0.11741   0.11738   0.11841  -1.7054    0.078291  0.078358  0.078382
 -0.83415 ]
-3.0214e-14
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLA

Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
[0.     0.     0.     0.6841 0.     0.     0.     0.    ]
[1.2444  1.2464  0.77198 2.3867  2.1875  2.1875  1.7014  0.97223]
0.10853
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
[0. 0. 0. 0. 0. 0. 0. 0.]
[ 1.2021   1.2259   0.85548 -0.55614  1.8017   2.1875   1.9445   0.48612]
0.093576
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
[0.      0.      0.      0.13903 0.      0.      0.      0.     ]
[1.2018  1.2186  0.6631  1.319   1.9306  2.1875  1.4583  0.97223]
0.096917
We will compute day: 20181024
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATL

Waiting for MATLAB...
[0.38187   0.        0.        0.        0.0070767 0.60477   0.28047
 0.4329   ]
[2.2222  0.55541 1.2961  1.1108  0.39774 2.1894  2.1894  2.6759 ]
0.03549
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
[0.       0.       0.       0.       0.       0.057417 0.       0.      ]
[-0.94616 -0.9204  -0.16653 -1.0666  -1.0907   0.13822 -0.81817 -1.2651 ]
4.4481e-05
We will compute day: 20181111
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
[0. 0. 0. 0. 0. 0. 0. 0.]
[-0.94616 -1.1055  -0.16646 -1.0665  -2.0637  -1.8087  -0.8192  -1.2663 ]
-4.0325e-14
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...
Waiting for MATLAB...


KeyboardInterrupt: 